In [ ]:
import xarray as xr
import pystac_client
import itertools
import time
import numpy as np
import metview as mv
import requests
from urllib.parse import urlparse
import os
os.environ['PROJ_LIB'] = "/opt/conda/envs/geospatial/share/proj"

xr.set_options(keep_attrs=True)

<span style="color:#336699; font-size:24px;font-weight: bold;">hands on - WMO - World Meteorological Organization </span>
<hr style="border:2px solid #0077b9;">

<br/>

<div style="text-align: center;font-size: 90%;">
    Alex de Almeida Fernandes - <a href="mailto:alex.fernandes@inpe.br">alex.fernandes@inpe.br</a><br/>
        Felipe Odorizi de Mello - <a href="mailto:felipe.odorizi@inpe.br">felipe.odorizi@inpe.br</a><br/><br/>
    DIPTC – Weather and Climate Forecast Division
    <br/>
    CGCT – General Coordination of Earth Sciences
    <br/>
    INPE – National Institute for Space Research
    <br/>
    Brazil
    <br/><br/>
    Last Updated: November 11, 2025
</div>


<br/>

<div style="text-align: center;font-size: 18px;">
<b>hands on - WMO - World Meteorological Organization </b></div>


# 🌡️ SAMeT
<hr style="border:1px solid #0077b9;">

**SAMeT (South American Mapping of Temperature)** is a product available for South America through the National Institute for Space Research
(CPTEC / INPE) Weather Forecast and Climate Studies website. This product combines the observed data with the ERA5 reanalysis, making a correction of the temperature field using an estimated lapse rate.

**The Lapse Rate** was estimated for four regions located over South America, and for the four seasons of the year. The estimated values for the Lapse Rate were, in all situations, lower than the standard Lapse Rate (-6.5 °C·km-¹ ). The combination of reanalysis and observations, together with the correction of the Lapse Rate produced more accurate temperature fields than those provided by ERA5, especially in regions with more accentuated topography more details in
https://ftp.cptec.inpe.br/modelos/tempo/SAMeT/Rozante_et_al_2021.pdf .

## 👩🏽‍💻 STAC Client API
<hr style="border:1px solid #0077b9;">

To run the example in this Jupyter Notebook, the package will be installed. [pystac-client](https://pystac-client.readthedocs.io/en/latest/).

In [ ]:
service = pystac_client.Client.open('https://data.inpe.br/bdc/stac/v1/')
service

## 🗂 STAC get_collections()
<hr style="border:1px solid #0077b9;">

The method for retrieving all collections from a STAC service.

In [ ]:
for collection in service.get_collections():
    print(f'🌡️\033[94m{collection.id}') if collection.id.find('samet') != -1 else print (f'\033[0m{collection.id}')

## 🔍 STAC Filter - get_collection()
<hr style="border:1px solid #0077b9;">

Is used to retrieve detailed metadata about a specific STAC Collection when you know its unique ID. 

In [ ]:
collection = service.get_collection('samet_daily-1')
collection

## 📂 STAC get_items()
<hr style="border:1px solid #0077b9;">

The Collection.get_items() method in the PySTAC library is used to retrieve STAC Items associated with a specific STAC Collection.
This method provides a way to access the individual spatiotemporal resources.

In [ ]:
from itertools import islice
item = next(islice(itertools.islice(collection.get_items(), 1), 1))

print("ID:", item.id)
print("Datetime:", item.datetime)
print("Bounding Box:", item.bbox)
print("Assets:")
for asset_key, asset in item.assets.items():
    print(f"  - {asset_key}: {asset.href}")


## 📄 STAC item
<hr style="border:1px solid #0077b9;">

A **STAC** (SpatioTemporal Asset Catalog) Item is a JSON file that acts as an atomic unit of metadata, describing a single geospatial asset. Its content *includes a GeoJSON feature with additional fields for time, links to related entities, and most importantly, links to the actual data assets it represents, such as images, lidar, or other files*. This makes it a foundational building block for discovering and cataloging spatiotemporal data. 

**Key components of a STAC Item**

***GeoJSON Feature***: The base of the item is a GeoJSON object, providing the spatial extent and geometry of the asset.
Assets: A STAC Item contains a collection of assets, which are links to the actual data files (e.g., raster images, point clouds, or other data). For example, a single satellite scene's different bands (like red, green, blue) would each be a separate asset within a single STAC Item.

***Metadata***: Additional fields are added to the GeoJSON to describe the asset, such as:
Time: The temporal extent of the data.

***Links***: URLs to related items or collections.

***Thumbnails***: Links to smaller preview images for quick browsing.

***Extensibility***: The STAC specification is built with a minimal core and a flexible extension mechanism, allowing it to support a wide variety of data types beyond just imagery. 

In [ ]:
item

In [ ]:
item.assets

In [ ]:
url = item.assets['tmax'].href
url

## ⬇️ Retrieving SAMet file from URL.
<hr style="border:1px solid #0077b9;">

In [ ]:
tmp_dir = "Dados/tmp/"
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)
else:
    for file in os.listdir(tmp_dir):
        path = os.path.join(tmp_dir, file)
        if os.path.isfile(path):
            os.remove(path)
        elif os.path.isdir(path):
            os.rmdir(path)

In [ ]:
response = requests.get(url)
filename = os.path.basename(urlparse(url).path)
output_path = os.path.join(tmp_dir, filename)

with open(output_path, 'wb') as f:
    f.write(response.content)

nc = mv.read(output_path)


<div style="display: flex; flex-direction: column; align-items: center;">
  <img src="https://metview.readthedocs.io/en/latest/_images/NETCDFVIS.png" alt="NetCDF Visualiser">
  <span style="color:#000000; font-size:16px; padding: 12px;">netcdf_visualiser - Defines the visualisation of NetCDF data
      allowing to specify which variables in the file should be used, and how to interpret them. The generated object can directly be used in plot().
  <a href="https://metview.readthedocs.io/en/latest/gen_files/icon_functions/netcdf_visualiser.html">(details)</a>
  </span>
</div>
<hr style="border:1px solid #0077b9;">

In [ ]:
mv.setoutput("jupyter")

In [ ]:
tmax =  mv.netcdf_visualiser(
    netcdf_plot_type="geo_matrix",
    netcdf_latitude_variable="lat",
    netcdf_longitude_variable="lon",
    netcdf_value_variable="tmax",
    netcdf_data=nc,
)

In [ ]:
area = [-60,-120,20,0] # S,W,N,E
AMS = mv.geoview(
    map_area_definition = "corners",
    area                = area,
    coastlines = mv.mcoast(
        map_coastline_land_shade        = "on",
        map_coastline_land_shade_colour = "#eeeeee",
        map_grid_latitude_increment     = 10,
        map_grid_longitude_increment    = 10)
    )

auto_style = mv.mcont(contour_automatic_setting = "ecmwf")

shaded = mv.mcont(
    legend                       = "on",
    contour_level_selection_type = "level_list",
    contour_level_list           = [-5, 0, 5, 10, 12.5, 15, 17.5, 20, 22.5, 25, 27.5, 30, 32.5, 35, 37.5, 40, 42.5, 45],
    contour_label                = "off",
    contour_line_colour_rainbow="ON",
    contour_highlight="OFF",
    contour_shade="ON",
    contour_shade_method="AREA_FILL"
    )

Coastlines_FG_ = mv.mcoast(
    map_boundaries="ON",
    map_boundaries_colour="BLACK",
    map_grid_colour="RGB(0,0,0)",
    map_label_colour="RGB(0,0,0)"
    )

_Legend_ = mv.mlegend(
    legend_text_font_size=0.3,
    legend_entry_border_colour="RGB(0,0,0)"
    )

_Text_Plotting_ = mv.mtext(
    text_font_size=0.6,
    text_lines="South American Mapping of Temperature (SAMeT)"
    )

mv.plot(AMS, tmax, shaded, Coastlines_FG_, _Legend_, _Text_Plotting_)


<img src="https://docs.xarray.dev/en/stable/_static/Xarray_Logo_RGB_Final.svg" align="right" width="100"/>
<span style="color:#336699; font-size:24px;font-weight: bold;">xarray</span>
<hr style="border:2px solid #0077b9;">
<img alt="_images/metview-all-in-one.png" src="https://docs.xarray.dev/en/v2022.09.0/_images/dataset-diagram.png" style="width: 300px;" />
<p><strong>Xarray </strong> provides data models for working with labeled arrays and datasets. Its toolkit includes a broad set of domain-agnostic functions for advanced analytics and visualization with these data structures. Interoperable with the scientific Python ecosystem including NumPy, Dask, Pandas, and Matplotlib.
</p>
<a  href="https://docs.xarray.dev/en/stable/index.html" title="Xarray">documentation</a>

In [ ]:
#!pip install netCDF4
#!pip install h5netcdf

In [ ]:
output_path

In [ ]:
import xarray as xr
ds = xr.open_dataset(output_path)
ds


In [ ]:
tmax = ds['tmax'].values
print(tmax.shape)
print(f"Max: {ds['tmax'].max().values} Min: {ds['tmax'].min().values} Sum: {ds['tmax'].sum().values}")
print(f"Mean: {ds['tmax'].mean().values} Std: {ds['tmax'].std().values} Median: {ds['tmax'].median().values}")
print(f"Quantiles: {ds['tmax'].quantile([0.25, 0.5, 0.75]).values}")